In [1]:
# import transformer_lens
# import transformer_lens.utils as utils
# from transformer_lens.hook_points import (
    # HookPoint,
# )  # Hooking utilities
from transformer_lens import HookedTransformer
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import datasets

torch.manual_seed(42)

Definimos o modelo Pythia usando a bilbioteca TransformerLens, que permite ter acesso às ativações internas de diversos modelos.

In [2]:
model = HookedTransformer.from_pretrained("EleutherAI/pythia-70m-deduped", device='cuda:0')

/home/flfp/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer


Podemos então fazera inferência e visualizar todas as camadas acessíveis através da variável cache

In [3]:
text = ["Hello, world!!!!!!!!!!!!!!, Hello, world!!!!!!!!!!!!!! Hello, world!!!!!!!!!!!!!!", "Hello,"]
tokens = model.to_tokens(text)
print(tokens)

logits, cache = model.run_with_cache(tokens)

print("Cache keys:", cache.keys())

tensor([[    0, 12092,    13,  1533, 45939, 18963,  4672,    13, 24387,    13,
          1533, 45939, 18963,  4672, 24387,    13,  1533, 45939, 18963,  4672],
        [    0, 12092,    13,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]],
       device='cuda:0')
Cache keys: dict_keys(['hook_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_rot_q', 'blocks.0.attn.hook_rot_k', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook

Podemos utilizar a biblioteca datasets do HuggingFace para importar um dataset pronto. Neste caso utilizamos o wikitext por ter um domínio bem definido e ser pequeno o suficiente para rodarmos localmente.

In [3]:
dataset = datasets.load_dataset("Salesforce/wikitext", split="train[:1801350]", ignore_verifications=True)

/home/flfp/.local/lib/python3.10/site-packages/datasets/load.py:1748: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
Found cached dataset parquet (/home/flfp/.cache/huggingface/datasets/Salesforce___parquet/wikitext-103-raw-v1-7bb180478b704b56/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Podemos então gerar um dicionário com as ativações da residual stream de todas as camadas dessa forma:

In [4]:
layers = list(range(1, 2))
activations = {}
for layer in layers:
    activations[layer] = []

def create_batch(batch):
    texts = [item["text"] for item in batch]
    tokens = model.to_tokens(texts)
    return tokens

dataloader = DataLoader(dataset, batch_size=4, shuffle=False, collate_fn=create_batch)


for text in tqdm(dataloader):
    text = text.to('cuda:0')
    _, cache = model.run_with_cache(text, remove_batch_dim=False)

    for layer in layers:
        key = f"blocks.{layer}.hook_resid_post"
        print(cache[key].shape)
        activations[layer].append(cache[key])

    # algum leak aqui, memória sobe sem parar

    # nem isso funciona :/
    del text
    del cache
    del _


activations


  0%|          | 6/450338 [00:00<2:32:03, 49.36it/s]

torch.Size([4, 178, 512])
torch.Size([4, 119, 512])
torch.Size([4, 333, 512])
torch.Size([4, 230, 512])
torch.Size([4, 257, 512])
torch.Size([4, 337, 512])
torch.Size([4, 355, 512])
torch.Size([4, 301, 512])
torch.Size([4, 80, 512])


  0%|          | 17/450338 [00:00<2:28:20, 50.59it/s]

torch.Size([4, 164, 512])
torch.Size([4, 84, 512])
torch.Size([4, 277, 512])
torch.Size([4, 230, 512])
torch.Size([4, 114, 512])
torch.Size([4, 239, 512])
torch.Size([4, 303, 512])
torch.Size([4, 196, 512])
torch.Size([4, 82, 512])


  0%|          | 33/450338 [00:00<2:04:05, 60.48it/s]

...

  0%|          | 1548/450338 [00:23<1:59:51, 62.40it/s]

torch.Size([4, 94, 512])
torch.Size([4, 214, 512])
torch.Size([4, 61, 512])
torch.Size([4, 191, 512])
torch.Size([4, 74, 512])
torch.Size([4, 148, 512])
torch.Size([4, 214, 512])
torch.Size([4, 268, 512])
torch.Size([4, 14, 512])
torch.Size([4, 235, 512])
torch.Size([4, 219, 512])


  0%|          | 1558/450338 [00:23<1:55:11, 64.94it/s]

torch.Size([4, 487, 512])
torch.Size([4, 276, 512])
torch.Size([4, 267, 512])
torch.Size([4, 273, 512])
torch.Size([4, 228, 512])


OutOfMemoryError: CUDA out of memory. Tried to allocate 648.00 MiB (GPU 0; 5.67 GiB total capacity; 4.90 GiB already allocated; 187.69 MiB free; 5.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
torch.cuda.empty_cache()